In [1]:
%cd ..

/home/steven/proj/lifematrix/TransformerLM


In [57]:
import spacy
import torch
from torchtext.datasets import Multi30k
from torchtext.vocab import build_vocab_from_iterator
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab, build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from torchtext.utils import download_from_url, extract_archive
from torch.utils.data import Dataset, DataLoader
import io
import time
import pickle

In [3]:
# def load_tokenizers():
#     spacy_de = spacy.load("de_core_news_sm")
    
#     spacy_en = spacy.load("en_core_web_sm")
    
#     return spacy_de, spacy_en

# def tokenize(text, tokenizer):
#     return [tok.text for tok in tokenizer.tokenizer(text)]

# def yield_tokens(data_iter, tokenizer, index):
#     for d in data_iter:
#         yield tokenizer(d[index])

# def build_vocabulary(spacy_de, spacy_en):
#     def tokenize_de(text):
#         return tokenize(text, spacy_de)
    
#     def tokenize_en(text):
#         return tokenize(text, spacy_en)

#     print("Build English Vocabulary ...")
#     train, val, test = Multi30k(language_pair=("de", "en"))
#     vocab_tgt = build_vocab_from_iterator(
#         yield_tokens(train+val+test, tokenize_en, index=1),
#         min_freq=2,
#         specials=["<s>", "</s>", "<blank>", "<unk>"]
#     )
    
    
#     print("Build German Vocabulary ...")
#     train, val, test = Multi30k(language_pair=("de", "en"))
#     vocab_src = build_vocab_from_iterator(
#         yield_tokens(train+val+test, tokenize_de, index=0),
#         min_freq=2,
#         specials=["<s>", "</s>", "<blank>", "<unk>"]
#     )
    

#     vocab_src.set_default_index(vocab_src["<unk>"])
#     vocab_tgt.set_default_index(vocab_tgt["<unk>"])
    
#     return vocab_src, vocab_tgt

# def load_vocab(spacy_de, spacy_en):
#     vocab_ckpt_fname = "checkpoints/vocab.pt"
#     if not os.path.exists(vocab_ckpt_fname):
#         vocab_src, vocab_tgt = build_vocabulary(spacy_de, spacy_en)
#         torch.save((vocab_src, vocab_tgt), vocab_ckpt_fname)
#     else:
#         vocab_src, vocab_tgt = torch.load(vocab_ckpt_fname)
    

In [40]:
def load_dataset():
    url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
    train_urls = ('train.en.gz', 'train.de.gz')
    val_urls = ('val.en.gz', 'val.de.gz')
    test_urls = ('test_2016_flickr.en.gz', 'test_2016_flickr.de.gz')

    train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
    val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
    test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]
    
    def create_pair(filepaths):
        pairs = []
        with open(filepaths[0], encoding="utf8") as f1, open(filepaths[1], encoding="utf8") as f2:
            pairs = [(line1, line2) for line1, line2 in zip(f1, f2)]
        return pairs
    
    train = create_pair(train_filepaths)
    val = create_pair(val_filepaths)
    test = create_pair(test_filepaths)
    
    return train, val, test
            
train, val, test = load_dataset()
data_all = train # + val + test
len(train), len(val), len(test), len(data_all)

(29000, 1014, 1000, 29000)

In [41]:
train[10]

('A ballet class of five girls jumping in sequence.\n',
 'Eine Ballettklasse mit fünf Mädchen, die nacheinander springen.\n')

In [69]:


en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')


def build_or_load_vocabs(data, spacy_de, spacy_en):
    
    def build_vocab(data_iter, tokenizer, index):
        tokens_iter = map(lambda x: tokenizer(x[index]), data)
        vocab = build_vocab_from_iterator(tokens_iter, 
                                          specials=['<unk>', '<blank>', '<bos>', '<eos>'])
        vocab.set_default_index(vocab["<unk>"])
        print(len(vocab))
        return vocab

    vocab_ckpt_fname = "checkpoints/vocab.pt"
    if (not os.path.exists(vocab_ckpt_fname)) or True:
        vocab_src = build_vocab(data, spacy_en, 0)
        vocab_tgt = build_vocab(data, spacy_de, 1)
        torch.save((vocab_src, vocab_tgt), vocab_ckpt_fname)
    else:
        vocab_src, vocab_tgt = torch.load(vocab_ckpt_fname)
    
    return vocab_src, vocab_tgt

vocab_src, vocab_tgt = build_or_load_vocabs(data_all, en_tokenizer, de_tokenizer)
len(vocab_src), len(vocab_tgt)

11399
18545


(11399, 18545)

In [55]:
vocab_src.get_stoi()["<bos>"]

2

In [ ]:
def collate_batch(
        batch,
        src_pipeline,
        tgt_pipeline,
        src_vocab,
        tgt_vocab,
        device,
        max_padding=128,
        pad_id=2
):
    src_bos_id = torch.tensor([vocab_src.get_stoi()["<bos>"]], device=device)
    src_eos_id = torch.tensor([vocab_src.get_stoi()["<eos>"]], device=device)
    
    tgt_bos_id = torch.tensor([vocab_tgt.get_stoi()["<bos>"]], device=device)
    tgt_eos_id = torch.tensor([vocab_tgt.get_stoi()["<eos>"]], device=device) 
    
    src_list, tgt_list = [], []
    for (_src, _tgt) in batch:
        processed_src = torch.cat(
            [
                src_bos_id,
                torch.tensor(src_vocab(src_pipeline(_src)), dtype=torch.int64, device=device),
                src_eos_id,
            ],
            dim=0
        )
        processes_tgt = torch.cat(
            [
                tgt_bos_id,
                torch.tensor(tgt_vocab(tgt_pipeline(_tgt)), dtype=torch.int64, device=device),
                tgt_eos_id,
            ],
            dim=0        
        )
        
        src_list.append(
            pad(processed_src, (0, max_padding-len(processed_src)), value=pad_id),
        )
        tgt_list.append(
            pad(processed_tgt, (0, max_padding-len(processed_tgt)), value=pad_id),
        )       
        
    src = torch.stack(src_list)
    tgt = torch.stack(tgt_lish)
    
    return (src, tgt)

In [65]:
def create_dataloaders(device, vocab_src, vocab_tgt, 
                       tokenizer_src, ttokenizer_tgt, 
                       batch_size=1200, max_padding=128):
    
    def collate_fn(batch):
        return collate_batch(
            batch, 
            lambda t: tokenizer_src(t),
            lambda t: ttokenizer_tgt(t),
            vocab_src, vocab_tgt,
            device, 
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()["<blank>"]
        )
    train, val, test = load_dataset()
    
    train_dataloader = DataLoader(
        train, batch_size=batch_size, shuffle=False
    )
    
    val_dataloader = DataLoader(
        val, batch_size=batch_size, shuffle=False
    )
    
    return train_dataloader, val_dataloader



In [ ]:
def train_model(device, vocab_src, vocab_tgt, tokenizer_src, tokenizer_tgt):
    torch.cuda.set_device(device)
    pad_idx = vocab_tgt["<blank>"]
    d_model = 512
    model = LMTransformerBilateralcoder(src_vocab_size=len(vocab_src),
                                        tgt_vocab_size=len(vocab_tgt),
                                        n_encoder_layers=2,
                                        dropout_rate=0.1).to(device)
    criterion = LabelSmoothing(
        size=len(vocab_tgt), padding_idx=pad_idx, smoothing=0.1
    ).to(device)
    
    train_dataloader, valid_dataloader = create_dataloaders(
        device,
        vocab_src,
        vocab_tgt,
        okenizer_src,
        tokenizer_tgt,
        batch_size=config["batch_size"] // ngpus_per_node,
        max_padding=config["max_padding"],
    )
    
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["base_lr"], betas=(0.9, 0.98), eps=1e-9
    )
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: lr_fn(
            step, d_model, factor=1, warmup=config["warmup"]
        ),
    )

    train_state = TrainState()

    for epoch in range(config["num_epochs"]):
        model.train()
        _, train_state = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in train_dataloader),
            model,
            SimpleLossCompute(module.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train+log",
            accum_iter=config["accum_iter"],
            train_state=train_state,
        )
        sloss = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in valid_dataloader),
            model,
            SimpleLossCompute(module.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )

In [31]:
for i in range(len(vocab_src)):
    print(i, vocab_src.lookup_token(i))

0 <unk>
1 <pad>
2 <bos>
3 <eos>
4 a
5 

6 .
7 A
8 in
9 the
10 on
11 is
12 and
13 man
14 of
15 with
16 ,
17 woman
18 are
19 to
20 Two
21 at
22 wearing
23 people
24 white
25 shirt
26 young
27 black
28 his
29 an
30 while
31 blue
32 sitting
33 red
34 girl
35 dog
36 boy
37 men
38 standing
39 playing
40 group
41 street
42 down
43 walking
44 front
45 her
46 holding
47 water
48 by
49 The
50 An
51 up
52 one
53 for
54 green
55 women
56 looking
57 outside
58 child
59 Three
60 as
61 large
62 little
63 through
64 yellow
65 two
66 brown
67 from
68 ball
69 their
70 hat
71 into
72 person
73 children
74 other
75 next
76 small
77 dressed
78 out
79 over
80 building
81 People
82 riding
83 running
84 near
85 around
86 another
87 jacket
88 some
89 sidewalk
90 field
91 beach
92 orange
93 crowd
94 jumping
95 pink
96 sits
97 stands
98 behind
99 table
100 grass
101 background
102 snow
103 stand
104 bike
105 city
106 girls
107 air
108 player
109 Man
110 looks
111 wall
112 dogs
113 camera
114 top
115 off
116 dres

In [32]:
for i in range(len(vocab_tgt)):
    print(i, vocab_tgt.lookup_token(i))

0 <unk>
1 <pad>
2 <bos>
3 <eos>
4 

5 .
6 Ein
7 einem
8 in
9 und
10 ,
11 mit
12 auf
13 Mann
14 einer
15 Eine
16 ein
17 der
18 Frau
19 eine
20 die
21 einen
22 Zwei
23 m
24 i
25 an
26 -
27 von
28 sich
29 dem
30 Mädchen
31 vor
32 zu
33 steht
34 Männer
35 sitzt
36 Hund
37 den
38 Junge
39 Straße
40 während
41 Gruppe
42 hält
43 spielt
44 Hemd
45 über
46 Personen
47 das
48 eines
49 weißen
50 blauen
51 Frauen
52 Menschen
53 roten
54 Kind
55 ist
56 neben
57 stehen
58 aus
59 am
60 sitzen
61 spielen
62 durch
63 Drei
64 bei
65 geht
66 trägt
67 fährt
68 um
69 Wasser
70 Kinder
71 kleines
72 macht
73 schwarzen
74 Person
75 springt
76 zwei
77 entlang
78 kleiner
79 Leute
80 gehen
81 etwas
82 seinem
83 großen
84 Oberteil
85 Shirt
86 grünen
87 Hand
88 junge
89 Mehrere
90 läuft
91 Der
92 sind
93 Fahrrad
94 für
95 Freien
96 Jungen
97 Hintergrund
98 Ball
99 anderen
100 Jacke
101 Luft
102 Strand
103 gelben
104 hat
105 Kleidung
106 T
107 junger
108 schaut
109 hinter
110 nach
111 andere
112 Gebäude
113 schwarz

In [35]:
vocab_src_a = set([vocab_src.lookup_token(i) for i in range(len(vocab_src))])
with open("/tmp/vocab_src_a.pkl", "wb") as f:
    pickle.dump(vocab_src_a, f)


In [17]:
a = [3,4,5]
a[-1:] = [9,8]
a

[3, 4, 9, 8]

In [12]:
a

[9, 8, 4, 5]

In [18]:
a = [3,4,5]
a[len(a):] = [9,8]
a

[3, 4, 5, 9, 8]

In [36]:
!ls -al checkpoints/

total 584
drwxrwxr-x  2 steven steven   4096 Jan  7 15:41  .
drwxrwxr-x 11 steven steven   4096 Jan  7 15:21  ..
-rw-rw-r--  1 steven steven    164 Jan  7 13:25 '=2.0.0'
-rw-rw-r--  1 steven steven     47 Jan  6 17:22  .gitignore
-rw-rw-r--  1 steven steven 577880 Jan  7 15:41  vocab.pt


In [58]:
ds_train = Dataset.from_list(train)

AttributeError: type object 'Dataset' has no attribute 'from_list'

In [59]:
dir(Dataset)

['__add__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_protocol']

In [63]:
dataloader = DataLoader(train, batch_size=8, shuffle=True)

In [64]:
for x in dataloader:
    print(x)
    break

[('Three young children play in a fountain.\n', 'Man walks his dog over interesting surface.\n', 'Three men are set up outside in front of a building, each playing an instrument.\n', 'A group of people are hiking in the forest.\n', 'A man running through a field with a number 41 marked on him.\n', 'A brown dog carries a long stick on the green grass.\n', 'A man in a green jacket smoking a cigarette throwing a small ball.\n', 'Men play saxaphones outside a building.\n'), ('Drei kleine Kinder spielen in einem Springbrunnen.\n', 'Ein Mann geht mit seinem Hund auf einem interessanten Untergrund.\n', 'Drei Männer haben sich vor einem Gebäude aufgestellt und jeder von ihnen spielt ein Instrument.\n', 'Eine Personengruppe wandert im Wald.\n', 'Ein Mann mit der Nummer\xa041 läuft über einen Platz.\n', 'Ein brauner Hund trägt einen langen Stock auf dem grünen Rasen.\n', 'Ein Mann in einer grünen Jacke raucht eine Zigarette und wirft einen kleinen Ball.\n', 'Männer spielen vor einem Gebäude Saxo

In [74]:
vocab_tgt["<unk10000>"]

0